In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import mlflow
mlflow.autolog()


In [3]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Filter out classes: 0,2,4,6,8
ind = np.where((y_train == 1)+(y_train == 3) + (y_train == 5) + (y_train == 7)++ (y_train == 9))
x_train, y_train = x_train[ind], y_train[ind]


# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (30508, 28, 28, 1)
30508 train samples
10000 test samples


/Users/mattiasakesson/Documents/projects/fedn/env/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Applying '+' to a non-numerical array is ill-defined. Returning a copy, but in the future this will error.
  if sys.path[0] == '':


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [5]:

batch_size = 128
epochs = 10

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train on train set (Includes classes 1,3,5,7,9)
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/10
215/215 [==============================] - 7s 32ms/step - loss: 0.3787 - accuracy: 0.8707 - val_loss: 0.0725 - val_accuracy: 0.9803
Epoch 2/10
215/215 [==============================] - 7s 32ms/step - loss: 0.0929 - accuracy: 0.9706 - val_loss: 0.0456 - val_accuracy: 0.9885
Epoch 3/10
215/215 [==============================] - 7s 32ms/step - loss: 0.0662 - accuracy: 0.9789 - val_loss: 0.0363 - val_accuracy: 0.9905
Epoch 4/10
215/215 [==============================] - 7s 32ms/step - loss: 0.0508 - accuracy: 0.9840 - val_loss: 0.0331 - val_accuracy: 0.9918
Epoch 5/10
215/215 [==============================] - 7s 31ms/step - loss: 0.0405 - accuracy: 0.9876 - val_loss: 0.0231 - val_accuracy: 0.9928
Epoch 6/10
215/215 [==============================] - 8s 36ms/step - loss: 0.0385 - accuracy: 0.9885 - val_loss: 0.0228 - val_accuracy: 0.9931
Epoch 7/10
215/215 [==============================] - 7s 31ms/step - loss: 0.0331 - accuracy: 0.9896 - val_loss: 0.0215 - val_accuracy: 0.9931

In [6]:
# Evaluate on test set (includes all 10 classes)
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 9.2734 - accuracy: 0.5039


[9.273356437683105, 0.5038999915122986]